In [1]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [1]:
!pip install simpletransformers
!pip install spacy==3.2.4

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 249 kB 7.4 MB/s 
     |████████████████████████████████| 4.2 MB 61.3 MB/s 
     |████████████████████████████████| 346 kB 62.1 MB/s 
     |████████████████████████████████| 6.6 MB 42.5 MB/s 
     |████████████████████████████████| 1.8 MB 38.7 MB/s 
     |████████████████████████████████| 10.1 MB 41.8 MB/s 
     |████████████████████████████████| 43 kB 2.5 MB/s 
     |████████████████████████████████| 1.2 MB 11.4 MB/s 
     |████████████████████████████████| 86 kB 5.7 MB/s 
     |████████████████████████████████| 596 kB 61.6 MB/s 
     |████████████████████████████████| 145 kB 45.2 MB/s 
     |████████████████████████████████| 181 kB 59.8 MB/s 
     |████████████████████████████████| 63 kB 1.6 MB/s 
     |████████████████████████████████| 1.1 MB 46.7 MB/s 
     |████████████████████████████████| 212 kB 58.3 MB/s 
     |███████████████████████████████

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 6.0 MB 8.1 MB/s 
     |████████████████████████████████| 42 kB 1.4 MB/s 
     |████████████████████████████████| 181 kB 78.1 MB/s 
     |████████████████████████████████| 10.1 MB 50.6 MB/s 
     |████████████████████████████████| 457 kB 71.0 MB/s 
     |████████████████████████████████| 660 kB 68.0 MB/s 
     |████████████████████████████████| 58 kB 7.5 MB/s 
  Attempting uninstall: typing-extensions
    Found existing installation: typing-extensions 4.2.0
    Uninstalling typing-extensions-4.2.0:
      Successfully uninstalled typing-extensions-4.2.0
  Attempting uninstall: catalogue
    Found existing installation: catalogue 1.0.0
    Uninstalling catalogue-1.0.0:
      Successfully uninstalled catalogue-1.0.0
  Attempting uninstall: srsly
    Found existing installation: srsly 1.0.5
    Uninstalling srsly-1.0.5:
      Successfully uninstalled srs

In [4]:
!wget  https://www.dropbox.com/s/cfbiksql5oej9a2/punct-dts.zip?dl=1 -O punct-dts.zip
!unzip -qq punct-dts.zip

--2022-05-25 23:01:18--  https://www.dropbox.com/s/4w6p9raq2iw92c4/punct-dts.zip?dl=1
Resolving www.dropbox.com (www.dropbox.com)... 162.125.1.18, 2620:100:6016:18::a27d:112
Connecting to www.dropbox.com (www.dropbox.com)|162.125.1.18|:443... connected.
HTTP request sent, awaiting response... 301 Moved Permanently
Location: /s/dl/4w6p9raq2iw92c4/punct-dts.zip [following]
--2022-05-25 23:01:18--  https://www.dropbox.com/s/dl/4w6p9raq2iw92c4/punct-dts.zip
Reusing existing connection to www.dropbox.com:443.
HTTP request sent, awaiting response... 302 Found
Location: https://uc7390f738a74b3b4b5748daa8fc.dl.dropboxusercontent.com/cd/0/get/Bl9W0_Yeo_xFy2XvJiGQnTkCBxyyZ3gMLnb6ju7MDgM5lPyK2D8PNHqz37NZt4f6-1sFT7uSnwBHenyaAZ7Jmil5yIYdAUNeR2JeFORJrCl9W0PZ2sImPGec73WjwhkbRGbdo-cG5-RgkV66DVL_1kvRi1W9PUOe3ibmIwX0_GyjgFVwpi4lnPm09csTOE954E0/file?dl=1# [following]
--2022-05-25 23:01:19--  https://uc7390f738a74b3b4b5748daa8fc.dl.dropboxusercontent.com/cd/0/get/Bl9W0_Yeo_xFy2XvJiGQnTkCBxyyZ3gMLnb6ju7MDg

In [6]:
!unzip punct-news.zip

Archive:  punct-news.zip
   creating: news/
  inflating: news/dev.csv            
  inflating: news/test.csv           
  inflating: news/train.csv          


In [7]:
import os
import pandas as pd
DATASET_NAME = 'news'
BASE_DIR = f'./{DATASET_NAME}/'

dataset = {
    
}

for filename in os.listdir(BASE_DIR):
  dataset[filename.replace('.csv', '')] = pd.read_csv(os.path.join(BASE_DIR, filename)).dropna()
 

In [8]:
dataset['train']

,sentence_id,labels,words
0,27,O,Câmara
1,27,O,deve
2,27,O,apresentar
3,27,O,parecer
4,27,O,final
...,...,...,...
34320,12960,O,a
34321,12960,I-PERIOD,5
34322,12960,I-COMMA,380
34323,12960,O,16


In [9]:
!pip install wandb

import wandb
wandb.login(key='8e593ae9d0788bae2e0a84d07de0e76f5cf3dcf4')

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


Failed to detect the name of this notebook, you can set it manually with the WANDB_NOTEBOOK_NAME environment variable to enable code saving.
wandb: W&B API key is configured. Use `wandb login --relogin` to force relogin
wandb: WARNING If you're specifying your api key in code, ensure this code is not shared publicly.
wandb: WARNING Consider setting the WANDB_API_KEY environment variable, or running `wandb login` from the command line.
wandb: Appending key for api.wandb.ai to your netrc file: /root/.netrc


True

In [10]:
from simpletransformers.ner import NERModel, NERArgs
import torch
import wandb


In [12]:

# Create a new run
project = "bert-large-punct"
# Connect an Artifact to the run
model_name = "neuralmind/bert-base-portuguese-cased"
#my_model_artifact = run.use_artifact(my_model_name)

# Download model weights to a folder and return the path
#model_dir = my_model_artifact.download()
train_args = {
    'evaluate_during_training': True,
    'logging_steps': 10,
    'num_train_epochs': 15,
    'evaluate_during_training_steps': dataset['train'].shape[0],
    'save_eval_checkpoints': False,
    'train_batch_size': 16,
    'eval_batch_size': 8,
    'overwrite_output_dir': True,
    'save_eval_checkpoints':False,
    'save_model_every_epoch':False,
    'save_steps':-1,
    'labels_list':dataset['train'].labels.unique().tolist(),
    'use_early_stopping':True,
    'wandb_project':project,
    'wandb_kwargs':{'name': DATASET_NAME}
}


model = NERModel(
    "bert",
    model_name,
    args=train_args,
    use_cuda=torch.cuda.is_available()
)
model.train_model(dataset['train'], eval_data=dataset['dev'])
result, model_outputs, wrong_preds = model.eval_model(dataset['test'])

pd.DataFrame.from_dict(result, orient='index').T.to_csv('test_result.csv', index=False,index_label=False)




wandb: Currently logged in as: tblima. Use `wandb login --relogin` to force relogin


Downloading:   0%|          | 0.00/647 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/418M [00:00<?, ?B/s]

Some weights of the model checkpoint at neuralmind/bert-base-portuguese-cased were not used when initializing BertForTokenClassification: ['cls.predictions.transform.LayerNorm.weight', 'cls.predictions.decoder.weight', 'cls.predictions.bias', 'cls.predictions.transform.dense.bias', 'cls.seq_relationship.bias', 'cls.seq_relationship.weight', 'cls.predictions.transform.dense.weight', 'cls.predictions.transform.LayerNorm.bias']
- This IS expected if you are initializing BertForTokenClassification from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertForTokenClassification from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
Some weights of BertForTokenClassification were not initialized from the model check

Downloading:   0%|          | 0.00/205k [00:00<?, ?B/s]

...

Running Evaluation:   0%|          | 0/34 [00:00<?, ?it/s]

  0%|          | 0/2 [00:00<?, ?it/s]

Running Evaluation:   0%|          | 0/166 [00:00<?, ?it/s]

Training loss,█▃▂▂▂▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁
eval_loss,▆▁▁▂▄▅▅▅▇▇▇▇███
f1_score,▁▇▇███▇████████
global_step,▁▁▁▁▂▂▂▂▂▃▃▃▃▃▃▄▄▄▄▄▅▅▅▅▅▆▆▆▆▆▆▇▇▇▇▇▇███
lr,▂▄████▇▇▇▇▇▆▆▆▆▆▅▅▅▅▅▄▄▄▄▄▄▃▃▃▃▃▂▂▂▂▂▁▁▁
precision,▁█▇▇▇▇▇▇██▇▇▇██
recall,▁▅▇██▇▇█▇▇▇▇▇▇▇
train_loss,█▆▂▁▁▂▁▁▁▁▁▁▁▁▁
Training loss,0.00046
eval_loss,0.20575
f1_score,0.82943


wandb: WARNING wandb.plots.* functions are deprecated and will be removed in a future release. Please use wandb.plot.* instead.


In [13]:
from collections import Counter
import pandas as pd
import re 
from collections import defaultdict
from spacy.matcher import Matcher
from spacy.tokens import Span
from tqdm import tqdm
import spacy

In [14]:
nlp = spacy.blank('pt')

In [15]:
def preprocess_data(dataframe):
     

    TOTAL = len(list(dataframe.groupby("sentence_id")))
    data = []
    for _, group in tqdm(dataframe.groupby("sentence_id"), total=TOTAL):
        text = " ".join(group.words)
        doc = nlp.make_doc(text)
        ents = []
        count_dict = defaultdict(lambda : 0)
        for i, label in enumerate(group.labels):
            if label != "O":
                matching_word = group.words.tolist()[i]
                span = Span(doc, i, i+1, label=label)
                ents.append((span.start_char, span.end_char, label.replace('I-','')))
                count_dict[matching_word] += 1
        ent = (text, {
            'entities': ents
        })
        data.append(ent)
    return data

In [16]:

TEST_DATA = preprocess_data(dataset['test'])

100%|██████████| 1328/1328 [00:00<00:00, 1413.02it/s]


In [17]:
from spacy.tokens import Doc
from spacy.training import Example
from tqdm.notebook import trange
#import spacy_transformers
import numpy as np
import torch

In [34]:
!zip bert-base-news.zip -r ./outputs/best_model/ 

  adding: outputs/best_model/ (stored 0%)
  adding: outputs/best_model/special_tokens_map.json (deflated 40%)
  adding: outputs/best_model/vocab.txt (deflated 52%)
  adding: outputs/best_model/training_progress_scores.csv (deflated 50%)
  adding: outputs/best_model/config.json (deflated 55%)
  adding: outputs/best_model/eval_results.txt (deflated 22%)
  adding: outputs/best_model/model_args.json (deflated 60%)
  adding: outputs/best_model/optimizer.pt (deflated 22%)
  adding: outputs/best_model/pytorch_model.bin (deflated 7%)
  adding: outputs/best_model/tokenizer_config.json (deflated 36%)
  adding: outputs/best_model/scheduler.pt (deflated 49%)
  adding: outputs/best_model/training_args.bin (deflated 48%)


In [ ]:
!cp bert-base-news.zip '/content/drive/MyDrive/bert-base-news.zip'

In [18]:
from simpletransformers.ner import NERModel, NERArgs
model_args = NERArgs()
model_args.labels_list = ["O", "COMMA", "PERIOD", "QUESTION"]

model = NERModel(
    "bert",
    "./outputs/best_model/",
    args=model_args,
    use_cuda=torch.cuda.is_available()
)

In [19]:
y_true = []
texts = []
for _, group in dataset['test'].groupby("sentence_id"):
        text = " ".join(group.words)
        texts.append(text)
        y_true.append(group.labels.apply(lambda label: label.replace("I-","") ).tolist())

In [20]:
predictions = model.predict(texts)

  0%|          | 0/2 [00:00<?, ?it/s]

Running Prediction:   0%|          | 0/166 [00:00<?, ?it/s]

In [21]:

y_pred = []
for i,pred in enumerate(predictions[0], 1):
  y_pred.append(list(map(lambda item:list(item.values())[0].replace("I-",""), pred)))
    

In [ ]:
text = "Eu gostaria de ir para casa agora mas eu não sei como"

In [ ]:
list(predictions[0][0][-1].values())[0]

In [ ]:
from spacy.tokens import Span

predictions_ner = predictions[0]
doc = nlp(texts[0])
doc.set_ents([Span(doc, i, i+1, "I-"+list(item.values())[0]) 
               for i, item in enumerate(predictions_ner[0]) 
                if list(item.values())[0] !="O"])
ents = list(doc.ents)
ents[0].label_

In [ ]:
doc[:2]

In [ ]:
examples = []
for i, (text, entities) in enumerate(TEST_DATA):

    doc = nlp(text)
    doc.set_ents([Span(doc, i, i+1, list(item.values())[0]) 
               for i, item in enumerate(predictions_ner[i]) 
                if list(item.values())[0] !="O"])
  
    example = Example.from_dict(doc, entities)
    examples.append(example)

In [ ]:
example

In [ ]:
entities

In [ ]:
TEST_DATA[:1]

In [ ]:
from spacy.scorer import Scorer
scorer = Scorer(nlp)
scores = Scorer.score_spans(examples, "ents")
print(scores["ents_f"])

In [ ]:
ents_per_type = scores.pop('ents_per_type')
pd.DataFrame(ents_per_type).to_latex()

In [ ]:
pd.DataFrame.from_dict(scores, orient='index').T.to_latex()